In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import pairwise_distances

In [2]:
data_path = './assets/data_dummy.xlsx'
data = pd.read_excel(data_path)

display(data.sample(n=20, random_state=5))

,user_id,item_id,rating,review
37,20,1006,1,Harga mahal tapi kualitas cucian sangat rendah...
42,7,1005,5,Harga yang tidak masuk akal untuk layanan laun...
74,20,1022,1,"Pakaian jadi menyusut setelah dicuci, tidak se..."
17,24,1020,3,Pakaian yang dikembalikan masih basah dan tida...
23,10,1005,4,Pakaian jadi berbau tidak sedap setelah dicuci...
10,10,1020,1,"Hasil cucian luntur, pakaian jadi rusak. Tidak..."
90,19,1017,3,"Hasil cucian tidak rapi, pakaian terlihat tida..."
45,5,1003,1,Pengeringan yang terlalu lama membuat pakaian ...
57,15,1016,5,Menggunakan bahan kimia yang menyebabkan irita...
6,14,1024,5,Harga terlalu tinggi untuk kualitas cucian yan...


In [3]:
# Function to create pivot table
def createPivot(data, fillVal = None):
  pivot_table = data.pivot_table(index='user_id', columns='item_id', values='rating')
  return pivot_table.fillna(fillVal) if fillVal is not None else pivot_table

In [4]:
random_state = 10
test_size = 0.30 

# Split data into train and test sets
train, test = train_test_split(data, test_size=test_size, random_state=random_state)
test = test[test['user_id'].isin(train['user_id'])]

# Create a pivot table
train_pivot = createPivot(train, 0)
test_pivot = createPivot(test, 0)

# Convert ratings to binary values
train_binary = train.copy()
train_binary['rating'] = train_binary['rating'].apply(lambda x: 0 if x >= 1 else 1)
train_binary = createPivot(train_binary, 1)

test_binary = test.copy()
test_binary['rating'] = test_binary['rating'].apply(lambda x: 1 if x >= 1 else 0)
test_binary = createPivot(test_binary, 0)

In [5]:
# Compute mean ratings
mean = np.nanmean(createPivot(train), axis=1)

# Subtract mean from train pivot table
train_subtracted_by_mean = (createPivot(train).T - mean).T
train_subtracted_by_mean.fillna(0, inplace=True)

# Calculate correlation between user
correlation = 1 - pairwise_distances(train_subtracted_by_mean, metric='cosine')
correlation[np.isnan(correlation)] = 0

correlation_df = pd.DataFrame(correlation)
correlation_df['user_id'] = train_subtracted_by_mean.index
correlation_df.set_index('user_id',inplace=True)
correlation_df.columns = train_subtracted_by_mean.index.tolist()

correlation_df.shape, train_pivot.shape

((25, 25), (25, 26))

In [6]:
# Compute predicted ratings
predicted_rating = np.dot(correlation, train_pivot)
final_rating = np.multiply(predicted_rating, train_binary)

final_rating

item_id,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,...,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.154701,0.138281,1.454627,0.000000,1.095445,0.000000,0.000000,0.000000,0.000000,0.981467,...,2.822180,0.000000,0.831216,0.000000,0.589256,1.643168,0.000000,-0.519701,1.443376,0.000000
3,0.000000,0.000000,-0.204210,-0.460044,0.243018,-2.027532,0.000000,0.000000,-0.161734,-0.000000,...,-1.100400,-1.099552,0.270435,0.000000,0.000000,0.000000,-0.956107,0.000000,-1.500248,0.000000
4,0.000000,-0.534749,-1.642917,-1.060660,-0.406333,-0.534749,-0.353553,0.000000,0.039528,0.158114,...,-2.199889,-1.069497,0.000000,0.000000,0.000000,0.000000,0.000000,0.039528,0.000000,0.000000
5,0.000000,-0.466767,-1.434055,-1.270853,0.000000,-0.466767,-0.308607,0.000000,0.034503,0.138013,...,-1.747704,-0.243468,0.000000,0.872872,0.000000,0.517549,-0.690066,0.034503,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,-1.527525,0.000000,0.000000,0.000000,1.500000,0.000000,0.000000,0.592927,0.000000,...,2.964635,0.000000,1.778781,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.909407
8,2.000000,-1.320104,0.935810,0.000000,0.000000,0.000000,0.000000,0.000000,0.237171,0.000000,...,2.862932,2.512873,1.246261,0.000000,1.020621,0.000000,0.000000,0.846032,0.000000,-1.452540
9,0.000000,0.000000,0.000000,0.000000,-0.316228,0.204124,0.000000,0.577350,0.000000,0.000000,...,0.046010,-0.632456,0.000000,0.000000,0.000000,0.000000,1.154701,0.000000,0.000000,0.000000
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
from sklearn.preprocessing import MinMaxScaler

# Filter user correlation only for users in the test set
correlation_test_df = correlation_df[correlation_df.index.isin(test.user_id)]
correlation_test_df = correlation_test_df[list(set(test.user_id))]

# Get test user predicted ratings
test_user_predicted_ratings = np.dot(correlation_test_df, test_pivot)
test_user_predicted_ratings *= test_binary
test_user_predicted_ratings = np.where(test_user_predicted_ratings > 0, test_user_predicted_ratings, np.nan)

# Scale the predicted ratings between 1 and 5
scaler = MinMaxScaler(feature_range=(1, 5))
test_user_predicted_ratings = scaler.fit_transform(test_user_predicted_ratings)

# Metric evaluation
mse = np.nanmean((createPivot(test) - test_user_predicted_ratings)**2)
rmse = np.sqrt(mse)
mae = np.nanmean(np.abs(createPivot(test) - test_user_predicted_ratings))

print("[Metric Evaluation]")
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)

[Metric Evaluation]
Mean Squared Error (MSE): 2.703201277587481
Root Mean Squared Error (RMSE): 1.6441415016924428
Mean Absolute Error (MAE): 1.0779109356454815


c:\Users\hendy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:464: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
c:\Users\hendy\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_data.py:465: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import string
import emoji

# Load the model and tokenizer
model = tf.keras.models.load_model("./assets/model_architecture.h5")
model.load_weights("./assets/model_weights.h5")

with open('./assets/tokenizer.pkl', 'rb') as f:
  tokenizer = pickle.load(f)

def preprocess(text):
  text = text.lower()
  text = re.sub(r"\d+", "", text)
  text = text.translate(str.maketrans("", "", string.punctuation))
  text = text.strip()
  text = re.sub('\s+', ' ', text)
  text = re.sub(r'[\u2070-\u209F\u00B2-\u00B3\u00B9-\u00BF\u02B0-\u036F\u1AB0-\u1AFF\u2090-\u2094]+', '', text)
  text = emoji.demojize(text)
  text = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
  text = text.strip()
  text = re.sub('\s+', ' ', text)
  text = re.sub(r'[^\u0000-\u007F\uD800-\uDBFF\uDC00-\uDFFF]+', '', text)
  return text

def predict(x):
  preprocessed_input_text = preprocess(x)
  trythis = tokenizer.texts_to_sequences([preprocessed_input_text])
  input_sequence = pad_sequences(trythis, maxlen=120)
  # Still on discussion (whether multiplied by 5 or not)
  predictions = model.predict(input_sequence) * 5
  return predictions[0][0]

def getReviewData(x):
  product_sentiments = data[data['item_id'] == x]['review']
  predicted_sentiments = product_sentiments.apply(predict)
  sentiment_score = predicted_sentiments.mean()
  return sentiment_score

def rankRecommendation(final_rating, user_id, data, no_rec):
  user_ratings = final_rating.loc[user_id]
  recommendation = user_ratings.sort_values(ascending=False)[:no_rec]

  recommendation_table = pd.DataFrame(recommendation).reset_index().rename(columns={user_id: 'predicted_ratings'})
  recommendation_table.insert(0, 'user_id', user_id)
  # recommendation_table = recommendation_table[recommendation_table['predicted_ratings'] != 0]
  # display(recommendation_table)

  recommendation_table['sentiment_score'] = recommendation_table['item_id'].apply(getReviewData)\
  # Still on discussion sentiment_score multiplied by 2 or not
  recommendation_table['product_ranking_score'] = recommendation_table['predicted_ratings'] + (2 * recommendation_table['sentiment_score'])
  display(recommendation_table.sort_values(by='product_ranking_score', ascending=False).head(no_rec))

In [9]:
user_id = 12
no_rec = 15

recommendation_table = rankRecommendation(final_rating, user_id, data, no_rec)

C:\Users\hendy\AppData\Local\Temp\ipykernel_7720\1665846678.py:44: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  recommendation = user_ratings.sort_values(ascending=False)[:no_rec]


1/1 [==============================] - 0s 48ms/step


,user_id,item_id,predicted_ratings,sentiment_score,product_ranking_score
1,12,1009,1.118034,0.017175,1.152385
0,12,1012,1.130668,0.000011,1.130689
2,12,1001,1.047114,0.018143,1.083400
4,12,1017,0.752586,0.047875,0.848337
3,12,1005,0.806551,0.005255,0.817061
5,12,1023,0.279508,0.046862,0.373232
8,12,1024,0.000000,0.165260,0.330520
6,12,1008,0.279508,0.000003,0.279515
14,12,1007,0.000000,0.072410,0.144820
7,12,1000,0.000000,0.068174,0.136348
